In [1]:
!pip install sagemaker --upgrade
!pip install pytesseract transformers datasets evaluate sacrebleu sacremoses nltk tensorboard py7zr --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.5/706.5 kB 18.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.143.0-py2.py3-none-any.whl size=948773 sha256=45a907c0751a3f5c559d6eff7af8e273b98494b5e53d14733e65c9a2b19704b5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/84/6b/c7/7609620d9e78faf22307580cfe04dc6f8ec3cb1fad39f1d165
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.132.0
    Uninstalling sagemaker-2.132.0:
      Successfully uninstalled sagemaker-2.132.0
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 2.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 8.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 74.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.1/178.1 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 1.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 37.0 MB/s 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 35.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=92c0ee6c1cd53d8c74f3639722292ed14fed2247a04abdc4cc79e7493ba51efb
  Stored in directory: /home/ec2-user/.cache/pip/wheels/42/79/78/5ad3b042cb2d97c294535162cdbaf9b167e3b186eae55ab72d
Successfully built sacremoses


In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::442847032488:role/SagemakerTestRole
sagemaker bucket: sagemaker-eu-central-1-442847032488
sagemaker session region: eu-central-1


In [16]:
# experiment config
train_file = "translated.train.json"
validation_file = "translated.valid.json"
test_file = "translated.test.json"

affix = "translated"

model_id = "KBLab/bart-base-swedish-cased" # Hugging Face Model Id
save_dataset_path = "data" # local path to save processed dataset
src_column = "src" # column of input text is
dst_column = "dst" # column of the output text

In [4]:
data_files = {"train": train_file, "test": test_file}

from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

# Load dataset from the hub
dataset = load_dataset("json", data_files=data_files, field="data")
# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/json/default-21360711a5f6c007/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Train dataset size: 88837
Test dataset size: 100


In [5]:
max_sample_length = tokenizer.model_max_length
print(f"Max input length: {max_sample_length}")

Max input length: 1000000000000000019884624838656


In [6]:
from datasets import concatenate_datasets

tokenized_inputs = concatenate_datasets([dataset["test"], dataset["train"]]).map(lambda x: tokenizer(x[src_column], truncation=True), batched=True, remove_columns=[src_column, dst_column])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
max_source_length = min(max_source_length, max_sample_length)
print(f"Max source length: {max_source_length}")


Map:   0%|          | 0/88937 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Max source length: 287


In [7]:
import numpy as np

tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x[dst_column], truncation=True), batched=True, remove_columns=[src_column, dst_column])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
max_target_length = int(np.percentile(target_lenghts, 95))
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/88937 [00:00<?, ? examples/s]

Max target length: 51


In [8]:
print("Preprocess function...")
def preprocess_function(sample, padding="max_length"):
    inputs = sample[src_column]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample[dst_column], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Preprocess function...


In [9]:
# process dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=list(dataset["train"].features))


Map:   0%|          | 0/88837 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/processed/{affix}/train'
tokenized_dataset["train"].save_to_disk(training_input_path)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/processed/{affix}/test'
tokenized_dataset["test"].save_to_disk(test_input_path)


print("uploaded data to:")
print(f"training dataset to: {training_input_path}")
print(f"test dataset to: {test_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/88837 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-eu-central-1-442847032488/processed/translated/train
test dataset to: s3://sagemaker-eu-central-1-442847032488/processed/translated/test


In [11]:
from transformers import AutoModelForSeq2SeqLM

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [18]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("sari")

# helper function to postprocess text
def postprocess_text(sources, preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    sources = [source.strip() for source in sources]

    return sources, preds, labels

def compute_metrics(eval_preds):
    preds, labels, sources = eval_preds
    # print(f"lengths: sources={len(sources)}, preds={len(preds)}, labels={len(labels)}")
    # print("sources=", sources[:3])
    # print("preds=", preds[:3])
    # print("labels", labels[:3])
    if isinstance(preds, tuple):
        preds = preds[0]
        
    decoded_sources = tokenizer.batch_decode(sources, skip_special_tokens=True)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_sources, decoded_preds, decoded_labels = postprocess_text(decoded_sources, decoded_preds, decoded_labels)

    # print("decoded_sources=", decoded_sources[:3])
    # print("decoded_preds=", decoded_preds[:3])
    # print("decoded_labels", decoded_labels[:3])
    result = metric.compute(sources=decoded_sources, predictions=decoded_preds, references=decoded_labels)
    result = {k: round(v, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [19]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{affix}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{repository_id}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"./{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    include_inputs_for_metrics=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [20]:
# Start training
trainer.train(resume_from_checkpoint = True)

You are resuming training from a checkpoint trained with 4.27.1 of Transformers but your current version is 4.27.4. This is not recommended and could yield to errors or unwanted behaviors.
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0it [00:00, ?it/s]

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=55525, training_loss=0.0, metrics={'train_runtime': 107.6432, 'train_samples_per_second': 4126.457, 'train_steps_per_second': 515.824, 'total_flos': 7.59080774979072e+16, 'train_loss': 0.0, 'epoch': 5.0})

In [23]:
trainer.evaluate()

{'eval_loss': 1.3517887592315674,
 'eval_sari': 36.5294,
 'eval_gen_len': 19.56,
 'eval_runtime': 77.294,
 'eval_samples_per_second': 1.294,
 'eval_steps_per_second': 0.168,
 'epoch': 5.0}

In [22]:
tokenizer.save_pretrained(repository_id)
trainer.save_model(repository_id)